In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from utils.pca_utils import k_components
import numpy as np

In [11]:
# Load the dataset
df = pd.read_csv('dataset.csv')

# Ensure the timestamp column is in datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Convert the target column to string
target_column = 'What is your intake?'  # Replace with your actual target column name
df[target_column] = df[target_column].astype(str)
df = df[df[target_column].isin(['2021', '2024'])].reset_index(drop=True)

# Drop the 'Timestamp' column
df = df.drop('Timestamp', axis=1)

dummies

In [12]:
X = df.drop(target_column, axis = 1)
y = df[target_column]

cols = X.columns
X_dummies = pd.get_dummies(X, columns= cols).astype(int)

Overal Score calculation

In [13]:
# Initialize an empty list to hold the overall scores
overall_scores = []

# Loop through each row in the DataFrame
for index, row in X_dummies.iterrows():
    score = 0
    # Loop through each column and calculate the weighted sum
    for col in X_dummies.columns:
        # Extract the sub-question number from the column name (e.g., Q1_1 -> 1)
        sub_question = int(float(col.split('_')[1]))
        score += row[col] * sub_question  # Multiply the value by the sub-question number
    overall_scores.append(score)

# Add the overall scores as a new column to the DataFrame
X_dummies['Overall_Satisfaction'] = overall_scores

PCA

In [14]:
new_target = 'Overall_Satisfaction'
X, y = X_dummies.drop(columns=new_target), X_dummies[new_target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = k_components(new_target, X_dummies, 0.9)

pca = PCA(n_components=k)
X_pca = pca.fit_transform(X_scaled)

train_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(k)])
train_pca[new_target] = y.reset_index(drop=True)

k = 27


PC1 top features

In [15]:
pca_loadings = np.abs(pca.components_[0])  # Loadings for PC1 (absolute values)
features = X.columns  # Feature names

# Create a DataFrame for PCA loadings
pca_df = pd.DataFrame({
    "Feature": features,
    "PCA Loading": pca_loadings
}).sort_values(by="PCA Loading", ascending=False)

In [16]:
pca_df.head(3)

,Feature,PCA Loading
15,Are labs regularly maintained?_4,0.236799
35,"How well-maintained are the classrooms (e.g., ...",0.230675
43,Are the sports facilities adequately maintaine...,0.225500


In [17]:
# # Plot PCA Loadings
# plt.figure(figsize=(10, 30))
# plt.barh(pca_df["Feature"], pca_df["PCA Loading"], color='skyblue')
# plt.title("Feature Contributions to Overall Satisfaction (PCA - PC1)")
# plt.xlabel("PCA Loading")
# plt.ylabel("Features")
# plt.gca().invert_yaxis()  # Highest loading on top
# plt.tight_layout()
# plt.show()